# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [6]:
# import libraries

# basic data libraries
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

# scikit-learn modules for pipelining, transformation, model fitting and classification
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import classification_report

# nltk-modules for text processing, tokenizing and lemmatizing
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download relevant ntlk packages
nltk.download(["punkt", "stopwords", "wordnet"])

# pickle for python object serialization and storing
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
# load data from database
engine = create_engine('sqlite:///crisisresponse.db')
df = pd.read_sql_table('messages', engine)
X = df.loc[:,"message"]
Y = df.iloc[:,4:40]

### 2. Write a tokenization function to process your text data

In [8]:
def tokenize(text):
    """
    Tokenize, lemmatize, lower and remove punctuation of input text.

    Input arguments:
        text: Single string with input text 
              Example: 'For today:= this is, a advanced _ example #- String!'
              
    Output:
        output: List of processed string
                Example: ['today', 'advanced', 'example', 'string']
        
    """
    # set text to lower case and remove punctuation
    text = re.sub("[\W_]", " ", text)
    text= text.lower()

    # tokenize words 
    tokens = word_tokenize(text)
    
    # lemmatizer
    lemmatizer = WordNetLemmatizer()
    
    # init and remove stopwords
    stop_words = set(stopwords.words('english'))
    output = [lemmatizer.lemmatize(w) for w in tokens if not w in stop_words]

    return output

In [9]:
tokenize("For today:= this is, a advanced _ example #- String!")

['today', 'advanced', 'example', 'string']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

print("""
Shape of all elements: \n\t X_train: {}
\t X_test: {}
\t Y_train: {}
\t Y_test: {}""".format(X_train.shape,
                          X_test.shape,
                          Y_train.shape,
                          Y_test.shape))


Shape of all elements: 
	 X_train: (18219,)
	 X_test: (7809,)
	 Y_train: (18219, 36)
	 Y_test: (7809, 36)


In [12]:
# train pipeline
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
!pip3 install --upgrade pandas==1.0.4

Requirement already up-to-date: pandas==1.0.4 in /opt/conda/lib/python3.6/site-packages (1.0.4)


In [47]:
# Create Functions for testing
def flat_arr_df(two_d_data):
    """
    Flatten array/list of arrays/lists and dataframes to lists.

    Input arguments:
        two_d_data: dataframe or array/list of arrays/lists 
                    Example: [[1,2,3],[4,5,6],[7,8,9]]
              
    Output:
        flat_data: List of flattened Input
                   Example: [1,2,3,4,5,6,7,8,9]
    
    """

    if isinstance(two_d_data, (list, np.ndarray)):
        if isinstance(two_d_data[0], (list, np.ndarray)):
            flat_data = [item for sublist in two_d_data for item in sublist]
        else:
            print("Wrong datatype used, cannot flat this object")
            return ""
    elif isinstance(two_d_data, pd.DataFrame):
            flat_data = list(two_d_data.values.flatten())
    
    return flat_data

def return_multioutput_f1_prec_recall(Y_pred, Y_test, mac_avg=False):
    """
    Output classification report (f1, precision, recall) seperated for every category in
    Multi_Output_Data

    Input arguments:
        Y_pred: dataframe or array/list of arrays/lists 
                    Example: [[1,2,3],[4,5,6],[7,8,9]]
        
        Y_test: dataframe or array/list of arrays/lists 
                    Example: [[1,2,3],[4,5,6],[7,8,9]]
                    
        mac_avg: If True returns Dict with Category names and F1 Score
              
    Output:
        If mac_avg==False: prints precision, recall and f1-score
        If mac_avg==True: returns Dict with Category names and F1 Score
    
    """
    if mac_avg:
        mac_avg_dict = {}
        
    for pred_row, test_row, colname in zip(Y_pred.T, Y_test.to_numpy().T, Y_test.columns):
        if mac_avg:
            mac_avg_dict[colname] = classification_report(pred_row, test_row, output_dict=True)["macro avg"]["f1-score"]
        else:
            print("Report for Category: {}".format(colname))
            print(classification_report(pred_row, test_row))    
    
    if mac_avg:
        return mac_avg_dict
    
def return_flatted_f1_prec_recall(Y_pred, Y_test, mac_avg=False):
    """
    Output classification report (f1, precision, recall) for flatted prediction and test data.

    Input arguments:
        Y_pred: dataframe or array/list of arrays/lists 
                    Example: [[1,2,3],[4,5,6],[7,8,9]]
        
        Y_test: dataframe or array/list of arrays/lists 
                    Example: [[1,2,3],[4,5,6],[7,8,9]]
                    
        mac_avg: If True returns F1-Score
              
    Output:
        If mac_avg==False: prints precision recall and f1-score
        If mac_avg==True: returns F1-Score only
    
    """
    
    flat_Y_pred = flat_arr_df(Y_pred)
    flat_Y_test = flat_arr_df(Y_test)
    if mac_avg:
        return classification_report(flat_Y_pred, flat_Y_test, output_dict=True)["macro avg"]["f1-score"]
    else:
        print(classification_report(flat_Y_pred, flat_Y_test))

In [31]:
test_data_arr = [[1,2,3],[4,5,6],[7,8,9]]
test_data_df = pd.DataFrame(test_data_arr)
flat_arr_df(test_data_arr)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [32]:
flat_arr_df(test_data_df)

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [45]:
return_flatted_f1_prec_recall(test_data_arr,test_data_df)

             precision    recall  f1-score   support

          1       1.00      1.00      1.00         1
          2       1.00      1.00      1.00         1
          3       1.00      1.00      1.00         1
          4       1.00      1.00      1.00         1
          5       1.00      1.00      1.00         1
          6       1.00      1.00      1.00         1
          7       1.00      1.00      1.00         1
          8       1.00      1.00      1.00         1
          9       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         9



In [34]:
# Test real model prediction
Y_pred_01 = pipeline.predict(X_test)

In [35]:
# Evaluate over whole flattened prediction
return_flatted_f1_prec_recall(Y_pred_01,Y_test)

             precision    recall  f1-score   support

          0       0.99      0.96      0.97    264811
          1       0.54      0.82      0.65     16313

avg / total       0.96      0.95      0.95    281124



In [50]:
!conda update pandas -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.6.14
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - pandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    pandas-0.24.2              |   py36hf484d3e_0        11.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:        11.4 MB

The following NEW packages will be INSTALLED:

  python_abi         conda-forge/linux-64::python_abi-3.6-1_cp36m

The fo

In [53]:
import pandas as pd
print(pd.__version__)

0.23.3


In [52]:
# Evaluate every Category on it's own
cla_report_rfc_b = pd.Series(return_multioutput_f1_prec_recall(Y_pred_01, Y_test, mac_avg=True))
cla_report_rfc_b = cla_report_rfc_b.to_frame()

AttributeError: 'DataFrame' object has no attribute 'to_numpy'

In [19]:
cla_report_rfc_b.rename(columns={0:"F1 Score for RandomForestClassifier without Modification"}, inplace=True)
_ = cla_report_rfc_b.plot(kind="bar", figsize=(15,5))
_ = plt.title("Macro Averaged F1-Score for several Model Attempts")
_ = plt.ylabel("Macro Averaged F1-Score")
_ = plt.xlabel("Multi-Output Categories")

AttributeError: 'DataFrame' object has no attribute 'to_numpy'

### 6. Improve your model
Use grid search to find better parameters. 

In [54]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=100)))
    ])

pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf__estimator__random_state', 'clf__estimator__verbose', 'clf__estimator__

In [55]:
parameters = {
        'clf__estimator__min_samples_split': [2, 3, 4],
        'clf__estimator__n_estimators': [100, 200, 300],
        'clf__estimator__criterion': ['entropy', 'gini']
    }

cv =  GridSearchCV(pipeline, param_grid=parameters, verbose=3, n_jobs=10)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, Y_train)

Fitting 3 folds for each of 18 candidates, totalling 54 fits
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=100 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=200 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 
[CV] clf__estimator__criterion=entropy, clf__estimator__min_samples_split=2, clf__estimator__n_estimators=300 
[CV] clf__estimator__criterion=entropy, clf__estima

In [ ]:
cv.best_params_

In [ ]:
cv.best_estimator_

In [ ]:
Y_pred_02 = cv.predict(X_test)
return_flatted_f1_prec_recall(Y_pred_02,Y_test)

In [ ]:
cla_report_rfc_a = pd.Series(return_multioutput_f1_prec_recall(Y_pred_02, Y_test,mac_avg=True))
cla_report_rfc_a = cla_report_rfc_a.to_frame()

cla_report_rfc_a.rename(columns={0:"F1 Score for RandomForestClassifier with Optimized Parameters"}, inplace=True)
cla_report_rfc_a = pd.concat([cla_report_rfc_b, cla_report_rfc_a], axis=1)
_ = cla_report_rfc_a.plot(kind="bar", figsize=(15,5))
_ = plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))
_ = plt.title("Macro Averaged F1-Score for several Model Attempts")
_ = plt.ylabel("Macro Averaged F1-Score")
_ = plt.xlabel("Multi-Output Categories")

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

https://github.com/herrfeder/DataScientist/tree/master/Project_02_Disaster_Response_Pipelines

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(rfc_ct_pipe, open("rfc_ct_classifier.pkl", "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.